In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Load Base datasets and add a "Data" column for tracking source
base_2022 = pd.read_excel("Pvt Car Jan to Dec 22(22 Base).xlsx")
base_2022["Data"] = "2022_Base"

base_2023 = pd.read_excel("Pvt Car Jan'23 to  Dec 23 base Final.xlsx")
base_2023["Data"] = "2023_Base"

base_2024 = pd.read_excel("cleaned_2024_Base_dataset.xlsx")
base_2024["Data"] = "2024_Base"

# Find common columns across all Base datasets
common_base_columns = list(set(base_2022.columns) & set(base_2023.columns) & set(base_2024.columns))

# Merge all Base datasets while keeping only common columns
base_merged = pd.concat([df[common_base_columns + ["Data"]] for df in [base_2022, base_2023, base_2024]], ignore_index=True)


In [3]:
# Load PR datasets and add a "Data" column
pr_2022 = pd.read_csv("cleaned_2022_PR.csv")
pr_2022["Data"] = "2022_PR"

pr_2023 = pd.read_excel("cleaned_PR dataset.xlsx")  
pr_2023["Data"] = "2023_PR"

pr_2024 = pd.read_excel("cleaned_2024_PR_tie_up.xlsx")
pr_2024["Data"] = "2024_PR"

# Find common columns in PR datasets (excluding "Old Policy No" for now)
common_pr_columns = list(set(pr_2022.columns) & set(pr_2023.columns) & set(pr_2024.columns))

# Merge all PR datasets while keeping only common PR columns
pr_merged = pd.concat([df[common_pr_columns + ["Data"]] for df in [pr_2022, pr_2023, pr_2024]], ignore_index=True)

# Ensure PR dataset follows the Base structure (adds missing columns as NaN)
pr_merged = pr_merged.reindex(columns=common_base_columns + ["Data"])

# Add "Old Policy No" from PR 2023 (if it exists)
if "Old Policy No" in pr_2023.columns and "Policy No" in pr_merged.columns:
    pr_merged = pr_merged.merge(pr_2023[['Policy No', 'Old Policy No']], on='Policy No', how='left')

# Ensure "Old Policy No" column is present in Base dataset as well (Base doesn't have this column)
base_merged["Old Policy No"] = None 

# Merge PR with Base
final_merged = pd.concat([base_merged, pr_merged], ignore_index=True)

# Database connection details
db_username = 'postgres'
db_password = 'kaviyam123'
db_host = 'localhost'  
db_port = '5432'
db_name = 'postgres'

# Create a connection to PostgreSQL
engine = create_engine(f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# Write to PostgreSQL
final_merged.to_sql('appended_base_and_pr', engine, if_exists='replace', index=False, chunksize=100000)

print("Merged data successfully written to PostgreSQL.")

C:\Users\Admin\AppData\Local\Temp\ipykernel_1184\568712456.py:2: DtypeWarning: Columns (17,29,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  pr_2022 = pd.read_csv("cleaned_2022_PR.csv")


ValueError: cannot reindex on an axis with duplicate labels

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Load Base datasets and add "Data" column
base_2022 = pd.read_excel("Pvt Car Jan to Dec 22(22 Base).xlsx")
base_2022["Data"] = "2022_Base"

base_2023 = pd.read_excel("Pvt Car Jan'23 to  Dec 23 base Final.xlsx")
base_2023["Data"] = "2023_Base"

base_2024 = pd.read_excel("cleaned_2024_Base_dataset.xlsx")
base_2024["Data"] = "2024_Base"

# Find common columns across all Base datasets
common_base_columns = list(set(base_2022.columns) & set(base_2023.columns) & set(base_2024.columns))

# Ensure "Data" column is included
common_base_columns.append("Data")

# Merge all Base datasets while keeping only common columns
base_merged = pd.concat([df[common_base_columns] for df in [base_2022, base_2023, base_2024]], ignore_index=True)

# Load PR datasets and add "Data" column
pr_2022 = pd.read_csv("cleaned_2022_PR.csv")
pr_2022["Data"] = "2022_PR"

pr_2023 = pd.read_excel("cleaned_PR dataset.xlsx")  # Contains "Old Policy No"
pr_2023["Data"] = "2023_PR"

pr_2024 = pd.read_excel("cleaned_2024_PR_tie_up.xlsx")
pr_2024["Data"] = "2024_PR"

# Find common columns in PR datasets
common_pr_columns = list(set(pr_2022.columns) & set(pr_2023.columns) & set(pr_2024.columns))

# Ensure "Data" column is included
common_pr_columns.append("Data")

# Merge all PR datasets while keeping only common PR columns
pr_merged = pd.concat([df[common_pr_columns] for df in [pr_2022, pr_2023, pr_2024]], ignore_index=True)

# Ensure PR dataset follows the Base structure (adds missing columns as NaN)
pr_merged = pr_merged.reindex(columns=common_base_columns, fill_value=None)

# Add "Old Policy No" from PR 2023 (if it exists)
if "Old Policy No" in pr_2023.columns and "Policy No" in pr_merged.columns:
    pr_merged = pr_merged.merge(pr_2023[['Policy No', 'Old Policy No']], on='Policy No', how='left')

# Ensure "Old Policy No" column is present in Base dataset as well
base_merged["Old Policy No"] = None  # Assign NaN to Base records

# Merge PR with Base
final_merged = pd.concat([base_merged, pr_merged], ignore_index=True)

# Database connection details
db_username = 'postgres'
db_password = 'kaviyam123'
db_host = 'localhost'  
db_port = '5432'
db_name = 'postgres'

# Create a connection to PostgreSQL
engine = create_engine(f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# Write to PostgreSQL
final_merged.to_sql('appended_base_and_pr', engine, if_exists='replace', index=False, chunksize=100000)

print(" Merged data successfully written to PostgreSQL.")

C:\Users\Admin\AppData\Local\Temp\ipykernel_11776\3429173114.py:24: DtypeWarning: Columns (17,29,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  pr_2022 = pd.read_csv("cleaned_2022_PR.csv")


ValueError: cannot reindex on an axis with duplicate labels

In [2]:
import pandas as pd

# Load all datasets
base_2022 = pd.read_excel("Pvt Car Jan to Dec 22(22 Base).xlsx")
base_2023 = pd.read_excel("Pvt Car Jan'23 to  Dec 23 base Final.xlsx")
base_2024 = pd.read_excel("cleaned_2024_Base_dataset.xlsx")

pr_2022 = pd.read_csv("cleaned_2022_PR.csv")
pr_2023 = pd.read_excel("cleaned_PR dataset.xlsx")  # Contains "Old Policy No"
pr_2024 = pd.read_excel("cleaned_2024_PR_tie_up.xlsx")

# Function to check for duplicate column names
def find_duplicate_columns(df, dataset_name):
    duplicates = df.columns[df.columns.duplicated()].tolist()
    if duplicates:
        print(f" Duplicate columns found in {dataset_name}: {duplicates}")
    else:
        print(f" No duplicate columns in {dataset_name}")

# Check for duplicates in each dataset
find_duplicate_columns(base_2022, "Base 2022")
find_duplicate_columns(base_2023, "Base 2023")
find_duplicate_columns(base_2024, "Base 2024")
find_duplicate_columns(pr_2022, "PR 2022")
find_duplicate_columns(pr_2023, "PR 2023")
find_duplicate_columns(pr_2024, "PR 2024")

C:\Users\Admin\AppData\Local\Temp\ipykernel_11776\1363414320.py:8: DtypeWarning: Columns (17,29,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  pr_2022 = pd.read_csv("cleaned_2022_PR.csv")


 No duplicate columns in Base 2022
 No duplicate columns in Base 2023
 No duplicate columns in Base 2024
 No duplicate columns in PR 2022
 No duplicate columns in PR 2023
 No duplicate columns in PR 2024


In [3]:
# Find common columns across Base datasets
common_base_columns = list(set(base_2022.columns) & set(base_2023.columns) & set(base_2024.columns))

# Find common columns across PR datasets
common_pr_columns = list(set(pr_2022.columns) & set(pr_2023.columns) & set(pr_2024.columns))

# Check if any columns are repeated in both Base and PR datasets
duplicate_columns_in_merged = list(set(common_base_columns) & set(common_pr_columns))

if duplicate_columns_in_merged:
    print(f"⚠️ Warning: These columns exist in both Base and PR datasets: {duplicate_columns_in_merged}")
else:
    print("✅ No duplicate columns found between Base and PR datasets.")


⚠️ Warning: These columns exist in both Base and PR datasets: [' New Vertical', 'Fuel Type', 'GST', 'Vehicle IDV', 'MANUFACTURER/Make', 'Policy End Date', 'ENGINENUMBER', 'Applicable Discount with NCB', 'Total Premium Payable ', 'Before GST Add-on GWP', 'CHASSIS NUMBER', 'NCB % Previous Year', 'New Branch Name  2', 'Policy Start Date', 'Road Side Assistance', 'Product name ', 'Total TP Premium', 'Total OD Premium', 'Age', 'Policy No', 'Month', 'Reg no ']


In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Function to standardize column names (strip spaces, lowercase, remove double spaces)
def clean_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(r'\s+', ' ', regex=True)
    return df

# Load Base datasets and clean column names
base_2022 = clean_columns(pd.read_excel("Pvt Car Jan to Dec 22(22 Base).xlsx"))
base_2022["Data"] = "2022_Base"

base_2023 = clean_columns(pd.read_excel("Pvt Car Jan'23 to  Dec 23 base Final.xlsx"))
base_2023["Data"] = "2023_Base"

base_2024 = clean_columns(pd.read_excel("cleaned_2024_Base_dataset.xlsx"))
base_2024["Data"] = "2024_Base"

# Find common columns across all Base datasets
common_base_columns = list(set(base_2022.columns) & set(base_2023.columns) & set(base_2024.columns))

# Ensure "Data" column is included
if "data" not in common_base_columns:
    common_base_columns.append("data")

# Merge all Base datasets while keeping only common columns
base_merged = pd.concat([df[common_base_columns] for df in [base_2022, base_2023, base_2024]], ignore_index=True)

# Load PR datasets and clean column names
pr_2022 = clean_columns(pd.read_csv("cleaned_2022_PR.csv"))
pr_2022["Data"] = "2022_PR"

pr_2023 = clean_columns(pd.read_excel("cleaned_PR dataset.xlsx"))  # Contains "Old Policy No"
pr_2023["Data"] = "2023_PR"

pr_2024 = clean_columns(pd.read_excel("cleaned_2024_PR_tie_up.xlsx"))
pr_2024["Data"] = "2024_PR"

# Find common columns in PR datasets
common_pr_columns = list(set(pr_2022.columns) & set(pr_2023.columns) & set(pr_2024.columns))

# Ensure "Data" column is included
if "data" not in common_pr_columns:
    common_pr_columns.append("data")

# Handle conflicting columns: Keep only Base structure
conflicting_columns = list(set(common_base_columns) & set(common_pr_columns))
print(f"Warning: These columns exist in both Base and PR datasets: {conflicting_columns}")

# Remove duplicates from PR dataset by keeping only common columns
pr_merged = pd.concat([df[common_pr_columns] for df in [pr_2022, pr_2023, pr_2024]], ignore_index=True)

# Ensure PR dataset follows the Base structure (adds missing columns as NaN)
for col in common_base_columns:
    if col not in pr_merged.columns:
        pr_merged[col] = None  # Add missing columns with NaN

# Add "Old Policy No" from PR 2023 (if it exists)
if "old policy no" in pr_2023.columns and "policy no" in pr_merged.columns:
    pr_merged = pr_merged.merge(pr_2023[['policy no', 'old policy no']], on='policy no', how='left')

# Ensure "Old Policy No" column is present in Base dataset as well
base_merged["old policy no"] = None  # Assign NaN to Base records

# Merge PR with Base
final_merged = pd.concat([base_merged, pr_merged], ignore_index=True)

# Database connection details
db_username = 'postgres'
db_password = 'kaviyam123'
db_host = 'localhost'  
db_port = '5432'
db_name = 'postgres'

# Create a connection to PostgreSQL
engine = create_engine(f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# Write to PostgreSQL
final_merged.to_sql('appended_base_and_pr', engine, if_exists='replace', index=False, chunksize=100000)

print("Merged data successfully written to PostgreSQL.")

KeyError: "['data'] not in index"

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Function to standardize column names (strip spaces, lowercase, remove extra spaces)
def clean_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(r'\s+', ' ', regex=True)
    return df

# Load Base datasets and clean column names
base_2022 = clean_columns(pd.read_excel("Pvt Car Jan to Dec 22(22 Base).xlsx"))
base_2022["Data"] = "2022_Base"

base_2023 = clean_columns(pd.read_excel("Pvt Car Jan'23 to  Dec 23 base Final.xlsx"))
base_2023["Data"] = "2023_Base"

base_2024 = clean_columns(pd.read_excel("cleaned_2024_Base_dataset.xlsx"))
base_2024["Data"] = "2024_Base"

# Ensure "Data" column is properly included before finding common columns
for df in [base_2022, base_2023, base_2024]:
    if "Data" not in df.columns:
        df["Data"] = None  # Add missing "Data" column

# Find common columns across all Base datasets
common_base_columns = list(set(base_2022.columns) & set(base_2023.columns) & set(base_2024.columns))

# Ensure "Data" column is included correctly
if "Data" in base_2022.columns:
    common_base_columns.append("Data")  # Add "Data" column explicitly

# Merge all Base datasets while keeping only common columns
base_merged = pd.concat([df[common_base_columns] for df in [base_2022, base_2023, base_2024]], ignore_index=True)

# Load PR datasets and clean column names
pr_2022 = clean_columns(pd.read_csv("cleaned_2022_PR.csv"))
pr_2022["Data"] = "2022_PR"

pr_2023 = clean_columns(pd.read_excel("cleaned_PR dataset.xlsx"))  # Contains "Old Policy No"
pr_2023["Data"] = "2023_PR"

pr_2024 = clean_columns(pd.read_excel("cleaned_2024_PR_tie_up.xlsx"))
pr_2024["Data"] = "2024_PR"

# Find common columns in PR datasets
common_pr_columns = list(set(pr_2022.columns) & set(pr_2023.columns) & set(pr_2024.columns))

# Ensure "Data" column is included correctly
if "Data" in pr_2022.columns:
    common_pr_columns.append("Data")

# Merge all PR datasets while keeping only common PR columns
pr_merged = pd.concat([df[common_pr_columns] for df in [pr_2022, pr_2023, pr_2024]], ignore_index=True)

# Ensure PR dataset follows the Base structure (adds missing columns as NaN)
for col in common_base_columns:
    if col not in pr_merged.columns:
        pr_merged[col] = None  # Add missing columns with NaN

# Add "Old Policy No" from PR 2023 (if it exists)
if "old policy no" in pr_2023.columns and "policy no" in pr_merged.columns:
    pr_merged = pr_merged.merge(pr_2023[['policy no', 'old policy no']], on='policy no', how='left')

# Ensure "Old Policy No" column is present in Base dataset as well
base_merged["old policy no"] = None  # Assign NaN to Base records

# Merge PR with Base
final_merged = pd.concat([base_merged, pr_merged], ignore_index=True)

# Database connection details
db_username = 'postgres'
db_password = 'kaviyam123'
db_host = 'localhost'  
db_port = '5432'
db_name = 'postgres'

# Create a connection to PostgreSQL
engine = create_engine(f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# Write to PostgreSQL
final_merged.to_sql('appended_base_and_pr', engine, if_exists='replace', index=False, chunksize=100000)

print("Merged data successfully written to PostgreSQL.")

C:\Users\Admin\AppData\Local\Temp\ipykernel_13100\929413441.py:35: DtypeWarning: Columns (17,29,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  pr_2022 = clean_columns(pd.read_csv("cleaned_2022_PR.csv"))


InvalidIndexError: Reindexing only valid with uniquely valued Index objects